In [2]:
from pyspark.sql import SparkSession
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
spark = SparkSession.builder \
    .appName("MNIST_CNN_Spark") \
    .getOrCreate()

print("Spark avviato su:", spark.sparkContext.master)

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

In [ ]:

BATCH_SIZE = 128

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache().shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)


In [2]:




#  CNN 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- 5. Callback---
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,          # ferma se non migliora dopo 3 epoche
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,          # dimezza il learning rate
        patience=2,          # dopo 2 epoche senza miglioramento
        min_lr=1e-6
    )
]

# --- 6. Addestramento ---
history = model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_test,
    callbacks=callbacks,
    verbose=1
)

# --- 7. Valutazione finale ---
test_loss, test_acc = model.evaluate(ds_test)
print(f"\n✅ Accuratezza su test: {test_acc:.4f}")
print(f"📉 Loss finale: {test_loss:.4f}")

# --- 8. Salva il modello ---
model.save("/app/cnn_mnist_model.h5")
print("Modello salvato in /app/cnn_mnist_model.h5")

spark.stop()


ModuleNotFoundError: No module named 'tensorflow'